<a href="https://colab.research.google.com/github/nagu165/cad-model-generator/blob/X/train_qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "unsloth[colab-ampere] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
import torch
from unsloth import FastLanguageModel

# Model details
max_seq_length = 2048 # You can choose a longer sequence length if needed
dtype = None # 'None' for auto-detection. Float16 for T4/V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4-bit quantization to dramatically reduce memory usage

# Load the model and tokenizer from Hugging Face
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Coder-7B-Instruct-128K",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

AttributeError: module 'cadquery' has no attribute 'freecad_impl'

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Suggested values: 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Set to 0 for optimal Unsloth performance
    bias = "none",    # Set to "none" for optimal Unsloth performance
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

In [ ]:
from datasets import load_dataset

# Load the dataset for CadQuery code generation
dataset_name = "filapro/cad-recode"
dataset = load_dataset(dataset_name, split="train")

# We create a simple instruction-response format for the model to learn
# This helps the model understand how to respond to prompts after fine-tuning
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs      = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        # The EOS_TOKEN is crucial for telling the model when a response is complete
        text = f"""### Instruction:
{instruction}

### Response:
{output} {tokenizer.eos_token}"""
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # We turn this off for this dataset structure
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,  # Increase for better results, e.g., 100 or more
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

# Start the training process
print("Starting the fine-tuning process...")
trainer_stats = trainer.train()
print("Fine-tuning complete!")

In [ ]:
# Unsloth makes inference fast and memory-efficient
from transformers import pipeline
import torch

# Clear VRAM cache before inference
torch.cuda.empty_cache()

# Create a test prompt using the same format as the training data
prompt = """### Instruction:
Write a CadQuery script that creates a sphere with a radius of 5.

### Response:
"""

# Set up the text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16 # Use bfloat16 for modern GPUs
)

# Generate the response
result = pipe(prompt, max_new_tokens=100, num_return_sequences=1)

print("--- Prompt ---")
print(prompt)
print("\n--- Generated CadQuery Code ---")
# We only print the generated part of the text
generated_text = result[0]['generated_text']
print(generated_text.split("### Response:")[1].strip())